In [1]:
import time
import copy
import random
import os
import cv2
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pyciede2000 import ciede2000
from collections import defaultdict
from openpyxl import Workbook,load_workbook

# dwave
import dimod
import minorminer
from dwave.samplers import SimulatedAnnealingSampler,TabuSampler
from dwave.system import LeapHybridSampler,DWaveSampler,EmbeddingComposite,FixedEmbeddingComposite
import dwave.inspector
from dwave.preprocessing import ScaleComposite
from minorminer.busclique import find_clique_embedding
from pyqubo import Spin,Array,Placeholder,Constraint,Binary
from dwave.preprocessing import ScaleComposite

In [2]:
def Image_processing(img,row,column):
    # 開啟圖片
    image_path = "ori/"
    image = cv2.imread(image_path+img)
    
    # 定義切割的行數和列數
    # column將圖片分成3行，左右
    # row將圖片分成3列，上下
    piece = column*row
    part_img_xy_dir=[]#儲存圖片位置訊息
    part_img_xy_dir_num=[]
    edge_img_rgb={}#儲存圖片的RGB訊息
    
    # 獲取圖片的總高度和總寬度
    height, width, _ = image.shape
    
    # 計算每個子圖片的高度和寬度
    sub_height = height // column
    sub_width = width // row

    counter = 0
    # 切割圖片
    for i in range(column):
        for j in range(row):
            # 計算子圖片的左上角和右下角座標
            x1 = j * sub_width
            y1 = i * sub_height
            x2 = (j + 1) * sub_width
            y2 = (i + 1) * sub_height
            # 切割子圖片
            part_img_xy_dir.append([y1,y2,x1,x2])
            part_img_xy_dir_num.append([y1,y2,x1,x2])
            counter+=1

    # random.shuffle(part_img_xy_dir)

    ori_position = [part_img_xy_dir_num.index(i) for i in  part_img_xy_dir]

    # 儲存子圖片
    for i in range(len(part_img_xy_dir)):
        cv2.imwrite("mid4/sub"+str(i)+".jpg",image[part_img_xy_dir[i][0]:part_img_xy_dir[i][1],part_img_xy_dir[i][2]:part_img_xy_dir[i][3]])
        
    #儲存每個圖塊左右上下的L*a*b色彩空間訊息
    for i in range(len(part_img_xy_dir)):
        L_rgb = image[part_img_xy_dir[i][0]:part_img_xy_dir[i][1],part_img_xy_dir[i][2]:part_img_xy_dir[i][2]+1]
        R_rgb = image[part_img_xy_dir[i][0]:part_img_xy_dir[i][1],part_img_xy_dir[i][3]-1:part_img_xy_dir[i][3]]
        T_rgb = image[part_img_xy_dir[i][0]:part_img_xy_dir[i][0]+1,part_img_xy_dir[i][2]:part_img_xy_dir[i][3]]
        D_rgb = image[part_img_xy_dir[i][1]-1:part_img_xy_dir[i][1],part_img_xy_dir[i][2]:part_img_xy_dir[i][3]]
        L_rgb = cv2.cvtColor(L_rgb, cv2.COLOR_BGR2LAB).astype(np.float32)
        R_rgb = cv2.cvtColor(R_rgb, cv2.COLOR_BGR2LAB).astype(np.float32)
        T_rgb = cv2.cvtColor(T_rgb, cv2.COLOR_BGR2LAB).astype(np.float32)
        D_rgb = cv2.cvtColor(D_rgb, cv2.COLOR_BGR2LAB).astype(np.float32)
        temp_dict = {"L":L_rgb,"R":R_rgb,"T":T_rgb,"D":D_rgb}
        edge_img_rgb[str(i)]=temp_dict
    return edge_img_rgb,piece,ori_position

In [3]:
def Color_different(edge_img_rgb,column,row,no_puzzle):
    # 色彩差異計算(差異度表格)
    
    # LR[0][1] meaing 01,LR[1][0] meaing 10
    # TD[0][1] meaing
    # |0|
    # |1|
    max_dif = 0.0
    piece = column*row
    LR_num = [[1]*piece for _ in range(piece)]
    TD_num = [[1]*piece for _ in range(piece)]
    
    L_num,R_num,T_num,D_num = [],[],[],[]
    num_mid_list = []
    
    for i in range(0,piece-1):
        for i1 in range(i+1,piece):
            L_all,R_all,T_all,D_all = 0,0,0,0
            # 計算左右
            for i2 in range(len(edge_img_rgb["0"]["L"])):
                L1,L2=[],[]
                R1,R2=[],[]
                for i3 in range(0,3):
                    if i3 == 0:
                        L1.append(edge_img_rgb[str(i)]["L"][i2][0][i3]/255*100)
                        R1.append(edge_img_rgb[str(i1)]["R"][i2][0][i3]/255*100)
                        L2.append(edge_img_rgb[str(i)]["R"][i2][0][i3]/255*100)
                        R2.append(edge_img_rgb[str(i1)]["L"][i2][0][i3]/255*100)
                    else:
                        L1.append(edge_img_rgb[str(i)]["L"][i2][0][i3]-128)
                        R1.append(edge_img_rgb[str(i1)]["R"][i2][0][i3]-128)
                        L2.append(edge_img_rgb[str(i)]["R"][i2][0][i3]-128)
                        R2.append(edge_img_rgb[str(i1)]["L"][i2][0][i3]-128)
                L3 = ciede2000((L1[0],L1[1],L1[2]), (R1[0],R1[1],R1[2]))
                R3 = ciede2000((L2[0],L2[1],L2[2]), (R2[0],R2[1],R2[2]))
                L_all += float(L3['delta_E_00'])
                R_all += float(R3['delta_E_00'])
            # 計算上下
            for i2 in range(len(edge_img_rgb["0"]["T"][0])):
                T1,T2=[],[]
                D1,D2=[],[]
                for i3 in range(0,3):
                    if i3 == 0:
                        T1.append(edge_img_rgb[str(i)]["T"][0][i2][i3]/255*100)
                        D1.append(edge_img_rgb[str(i1)]["D"][0][i2][i3]/255*100)
                        T2.append(edge_img_rgb[str(i)]["D"][0][i2][i3]/255*100)
                        D2.append(edge_img_rgb[str(i1)]["T"][0][i2][i3]/255*100)
                    else:
                        T1.append(edge_img_rgb[str(i)]["T"][0][i2][i3]-128)
                        D1.append(edge_img_rgb[str(i1)]["D"][0][i2][i3]-128)
                        T2.append(edge_img_rgb[str(i)]["D"][0][i2][i3]-128)
                        D2.append(edge_img_rgb[str(i1)]["T"][0][i2][i3]-128)
                T3 = copy.deepcopy(ciede2000((T1[0],T1[1],T1[2]), (D1[0],D1[1],D1[2])))
                D3 = copy.deepcopy(ciede2000((T2[0],T2[1],T2[2]), (D2[0],D2[1],D2[2])))
                T_all += float(T3['delta_E_00'])
                D_all += float(D3['delta_E_00'])
            #儲存成矩陣形式，並歸一化
            # 138.1692548861247 為在預設下Ciede2000色差法下可能的最大值
            LR_num[i][i1] = (float(R_all)/len(edge_img_rgb["0"]["L"]))/138.1692548861247
            LR_num[i1][i] = (float(L_all)/len(edge_img_rgb["0"]["L"]))/138.1692548861247
            TD_num[i][i1] = (float(D_all)/len(edge_img_rgb["0"]["T"][0]))/138.1692548861247
            TD_num[i1][i] = (float(T_all)/len(edge_img_rgb["0"]["T"][0]))/138.1692548861247
            num_mid_list.append(LR_num[i][i1])
            num_mid_list.append(LR_num[i1][i])
            num_mid_list.append(TD_num[i][i1])
            num_mid_list.append(TD_num[i1][i])

    
    
    median = np.median(num_mid_list)
    num_mid_list.sort()
    max_diff = num_mid_list[-1]
    print(max_diff)

    for i in range(0,len(no_puzzle)):
        for i1 in range(0,piece):
            if i != i1:
                LR_num[no_puzzle[i]][i1],LR_num[i1][no_puzzle[i]],TD_num[no_puzzle[i]][i1],TD_num[i1][no_puzzle[i]] = median,median,median,median
                
    return LR_num,TD_num,median,max_diff

In [4]:
def Search_connect_hailtonian(puzzle,row,column,no_puzzle=[]):
    #得知 No.0拼圖片附近的兩個關聯，建立Hamiltonian,Constraint

    # no puzzle
    
    # 排除不可能的情況
    side_const = 0.0
    row_li = [(column-1)*row+i for i in range(0,row)]
    col_li = [(row-1)*(i+1)+i for i in range(0,column)]
    # print(row_li)
    # print(col_li)

    co = 0

    no_puzzles = []
    
    # Hamiltonian
    for p1 in range(0,piece):
        for p2 in range(0,piece):
            if p1 != p2:
                for i1 in range(0,piece):
                    if p1 not in no_puzzles and p2 not in no_puzzles:
                        if i1 not in col_li:
                            side_const += LR_num[p1][p2]*puzzle[p1][i1]*puzzle[p2][i1+1]
                            co+=1
                        if i1 not in row_li:
                            side_const += TD_num[p1][p2]*puzzle[p1][i1]*puzzle[p2][i1+row]
                            co+=1
    # print("co",co)
    
    # Constraint
    constant = 0.0
    for k in range(0,piece):
        cons1 = 0.0
        cons2 = 0.0
        for k1 in range(0,piece):
            if k not in no_puzzles:
                cons1 += puzzle[k][k1]
            if k1 not in no_puzzles:
                cons2 += puzzle[k1][k]
        if k not in no_puzzles:constant += Constraint((cons1-1)**2,label = "cons1"+str(k))
        if k1 not in no_puzzles:constant += Constraint((cons2-1)**2,label = "cons2"+str(k))

    
    return side_const,constant

In [5]:
def default_ising_beta_range(h, J,
                             max_single_qubit_excitation_rate = 0.01,
                             scale_T_with_N = True):
    if not 0 < max_single_qubit_excitation_rate < 1:
        raise ValueError('Targeted single qubit excitations rates must be in range (0,1)')


    # Approximate worst and best cases of the [non-zero] energy signal (effective field)
    # experienced per spin as function of neighbors: bias = h_i + sum_j Jij s_j:
    sum_abs_bias_dict = defaultdict(int, {k: abs(v) for k, v in h.items()})
    if sum_abs_bias_dict:
        min_abs_bias_dict = {k: v for k, v in sum_abs_bias_dict.items() if v != 0}
    else:
        min_abs_bias_dict = {}
    #This loop is slow, but is not a bottleneck for practical implementations of simulated annealing:
    for (k1, k2), v in J.items():
        for k in [k1,k2]:
            sum_abs_bias_dict[k] += abs(v)
            if v != 0: #This is slow, but this routine is not a bottleneck:
                if k in min_abs_bias_dict:
                    min_abs_bias_dict[k] = min(abs(v),min_abs_bias_dict[k])
                else:
                    min_abs_bias_dict[k] = abs(v)

    if not min_abs_bias_dict:
        warn_msg = ('All bqm biases are zero (all energies are zero), this is '
                    'likely a value error. Temperature range is set arbitrarily '
                    'to [0.1,1]. Metropolis-Hastings update is non-ergodic.')
        warnings.warn(warn_msg)
        return([0.1,1])

    max_effective_field = max(sum_abs_bias_dict.values(), default=0)

    if max_effective_field == 0:
        hot_beta = 1
    else:
        hot_beta = np.log(2) / (2*max_effective_field)

    if len(min_abs_bias_dict)==0:
        #Trivial problem
        cold_beta = hot_beta
    else:
        values_array = np.array(list(min_abs_bias_dict.values()),dtype=float)
        min_effective_field = np.min(values_array)
        if scale_T_with_N:
            #This is effective assuming finite precision, for pathological
            #or real valued problems this approximation may perform poorly
            number_min_gaps = np.sum(min_effective_field == values_array)
        else:
            number_min_gaps = 1
        cold_beta = np.log(number_min_gaps/max_single_qubit_excitation_rate) / (2*min_effective_field)

    return [hot_beta, cold_beta]

In [6]:
def Hamiltonian_qubo_caculate(side_H,const,g,num_reads,num_sweeps,ori_position,no_puzzle):
    H =  side_H + g*const
    model = H.compile()
    
    # Create QUBO
    #h,J,offset = model.to_ising()
    #beta = default_ising_beta_range(h, J)#Simulated
    bqm = model.to_bqm()

    time_start = time.time()
    
    # SA
    sa = SimulatedAnnealingSampler()
    sampleset = sa.sample(bqm, num_reads=num_reads, num_sweeps=num_sweeps)

    # tabu
    # sa = TabuSampler()
    # sampleset = sa.sample(bqm,tenure = 2,num_restarts = 3,num_reads = 150)
    
    time_mid = time.time()-time_start
    
    sampleset_p = sampleset
    
    # Decode solution
    decoded_samples = model.decode_sampleset(sampleset)
    best_sample = min(decoded_samples, key=lambda x: x.energy)

    # puzzle confirm
    puzzle_position = ["n" for _ in range(0,piece)]
    # print(puzzle_position)
    
    for i in best_sample.sample.keys():
        if best_sample.sample[i] == 1:
            # print(i)
            u=i.replace("]","").split("[")
            puzzle_position[int(u[2])] = copy.deepcopy(int(u[1]))
            if int(u[1]) in no_puzzle:
                puzzle_position[int(u[2])] = "n"

    print(best_sample.energy)
    if puzzle_position == ori_position:
        print()
        print("true")
        print()
        print('posi',puzzle_position)
        correct = True
    else:
        print()
        print("false")
        print()
        print('posi',puzzle_position)
        correct = False
    print()

    return best_sample.energy,sampleset_p,time_mid,correct

In [7]:
# fff = ['01','02','03','04','05','06','07','08','09']
fff = ['01']
row = 3
column = 3

num = 1000

# 定義資料夾與excel位置
# folder_name = "data/simulator_"+str(row)+"_"+str(column)+"re"+fff[0]

# sa
folder_name = "data/test/simulator_"+str(row)+"_"+str(column)+'_t'
ex_name = folder_name+'/simulator_2_time_'+str(row)+"_"+str(column)+".xlsx"

# tabu
# folder_name = "data/test/tabu_"+str(row)+"_"+str(column)
# ex_name = folder_name+'/tabu_time_'+str(row)+"_"+str(column)+".xlsx"

if not os.path.exists(folder_name):
    os.makedirs(folder_name)
if os.path.exists(ex_name):
    wb = load_workbook(ex_name)
else:
    wb = Workbook()

ws = wb.active
ws['A2'] = 'num reads'
ws['A3'] = 'num sweeps'
ws['A4'] = "constraint puzzle median value"
ws['A5'] = "constraint extra add"
ws['A6'] = "constraint fixed"

ws['A8'] = 'prepar time'
ws['A9'] = 'calculate time'
ws["A10"] = "energy"
ws['A11'] = 'correct'
count = 1
for xfd in fff:
    time_start=time.time()
    
    print(xfd)
    no_puzzle = [8]
    
    edge_img_rgb,piece,ori_position = Image_processing(xfd+".jpg",row,column)
    LR_num,TD_num,median,max_diff = Color_different(edge_img_rgb,column,row,no_puzzle)
    time_color = time.time()
    print('Color time : ',time_color-time_start)
    #Binary變數列表
    puzzle  = Array.create("puzzle",(piece,piece),"BINARY")
    
    # 得到 Hailtonian、Constraint、使用了那些拼圖片
    side_H,const= Search_connect_hailtonian(puzzle,row,column)

    print("----------insert----------")
    #返回確認能量
    x = 0.0
    
    for i in range(0,column):
        for i1 in range(0,row-1):
            x_temp = LR_num[i1+row*i][i1+row*i+1]
            x += x_temp
            if i==0 and i1==0:
                x_min = x_temp
                x_max = x_temp
            x_min = x_temp if x_temp < x_min else x_min
            x_max = x_temp if x_temp > x_max else x_max
            # print(x_temp)
    for i in range(0,row):
        for i1 in range(0,column-1):
            x_temp = TD_num[i+row*i1][i+row+row*i1]
            x += x_temp
            x_min = x_temp if x_temp < x_min else x_min
            x_max = x_temp if x_temp > x_max else x_max
            # print(x_temp)

    print()
    print("min : ",x_min)
    print("max : ",x_max)
    print("total : ",x)

    # constraint control
    # con_va = median+0.04
    con_va_ex = 0.03
    con_va = median+con_va_ex
    
    con_fix = 0

    if con_fix != 0:
        con = con_fix
        con_va_ex = 0
        con_va = 0
    else:
        con = con_va
    
    print("constraint value : ",con_fix)
    print("----------insert----------")
    print("Stander energy : ",x)
    # time
    time_mid=time.time()
    time_mid_a = time_mid-time_start
    print("Finish-Preparatory work")
    print('Prepar time : ',time_mid_a)
    print()

    energy,sampleset_p,time_mid,correct = Hamiltonian_qubo_caculate(side_H,const,con,num,num,ori_position,no_puzzle)
    
    print("Calculate time : ",time_mid)
    print("energy : ",energy)
    print()

    # 返回性確認

    # excel
    ws.cell(row=1,column=int(count)+1,value=xfd)
    ws.cell(row=2,column=int(count)+1,value=num)
    ws.cell(row=3,column=int(count)+1,value=num)
    ws.cell(row=4,column=int(count)+1,value=median)
    ws.cell(row=5,column=int(count)+1,value=con_va_ex)
    ws.cell(row=6,column=int(count)+1,value=con_fix)
    
    ws.cell(row=8,column=int(count)+1,value=time_mid_a)
    ws.cell(row=9,column=int(count)+1,value=time_mid)
    ws.cell(row=10,column=int(count)+1,value=energy)
    ws.cell(row=11,column=int(count)+1,value=correct)

    df = sampleset_p.to_pandas_dataframe()
    # # 將 DataFrame 保存為文本文件
    df.to_excel(folder_name+'/Simulator_sampleset_'+xfd+".xlsx", index=False)
    
    print()
    print("median",median)
    print('*'*25)
    wb.save(ex_name)
    count += 1

01
0.29194280244583115
Color time :  4.11482834815979
----------insert----------

min :  0.0018895858608712075
max :  0.14492040921416086
total :  0.36608058133782767
constraint value :  0
----------insert----------
Stander energy :  0.36608058133782767
Finish-Preparatory work
Prepar time :  4.132203817367554

0.36608058133782784

false

posi [0, 1, 2, 3, 4, 5, 6, 7, 'n']

Calculate time :  0.9235982894897461
energy :  0.36608058133782784


median 0.14492040921416086
*************************
